In [1]:
# setting the environment variables, the keys
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

%load_ext autoreload
%autoreload 2

In [2]:
print(os.getcwd())
print(os.listdir())

/home/rehmanm/learning/python-code/langchain/book/chapter-1/src/chapter_1
['__pycache__', 'chat_model.ipynb', 'config.py', 'fake_llm.ipynb', '.ipynb_checkpoints', 'lcel.ipynb', '__init__.py']


In [3]:
from config import set_environment

set_environment()

In [4]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage 

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [6]:
llm = GoogleGenerativeAI(model="gemini-2.0-flash")
output_parser = StrOutputParser()

In [7]:
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")

chain = prompt | llm | output_parser

result = chain.invoke({"topic" : "programming"})
print(result)

Why do programmers prefer dark mode?

Because light attracts bugs!


## complex lcel example

In [8]:
story_prompt = PromptTemplate.from_template("Write a story about {topic}")
story_chain = story_prompt | llm | output_parser

story = story_chain.invoke("A rainy day")

In [9]:
analysis_prompt = PromptTemplate.from_template("Anlalyze the following story mood: {story}")
analysis_chain = analysis_prompt | llm | output_parser

analysis =analysis_chain.invoke({"story": story})

print(analysis)

The story's mood is predominantly **melancholy, hopeful, and comforting.** Here's a breakdown:

*   **Melancholy:** The story begins with a sense of dreariness and gloom established by the description of the rainy day. Words like "grey wash," "blurred," "seeped into your bones," and "relentless percussion" evoke a feeling of sadness and quiet despair. Elara's lateness and huddling in her worn raincoat contribute to this feeling of isolation and perhaps a hint of hardship.
*   **Hopeful:** Despite the initial gloom, the story offers a glimmer of hope. The "familiar glow" of Mrs. Hawthorne's window, the "warm, amber light," and the "welcome melody" of the bell create a sense of anticipation and promise.
*   **Comforting:** The description of Mrs. Hawthorne and her shop establishes a feeling of warmth and security. Words like "warm, comforting rumble," "haven," "gentle symphony of time," and "chamomile tea" create a cozy and inviting atmosphere. The act of Elara being cared for by Mrs. Ha

In [16]:
output_prompt = PromptTemplate.from_template("Here is the story \n{story} \n and it is the mode \n{mode}")
story_with_analysis = story_chain | analysis_chain 

print(story_with_analysis.invoke({"topic": "A rainy day"}))

The story's mood is predominantly **melancholy and somber, gradually shifting towards hopeful and heartwarming.**

Here's a breakdown of the mood elements:

*   **Beginning:** The opening paragraphs establish a strong sense of gloom, sadness, and isolation. Words like "smudges of grey," "mournful song," "lashed," "indistinct," "stale," "huddled," "overcast," "metallic tang of rain" all contribute to this mood. Elsie's loneliness and the description of the weather mirroring her emotional state reinforce the initial feeling of sadness.
*   **Middle:** The mood begins to lighten as Elsie finds solace in baking. The description of the baking process is calming and comforting: "rhythm of the kitchen," "whir of the mixer," "gentle clinking of bowls," "rising warmth of the oven." These details offer a glimmer of hope and a sense of purpose. The introduction of Clara, also lost and vulnerable, introduces an opportunity for connection.
*   **End:** The turning point is the shared experience of 

In [20]:
from langchain_core.runnables import RunnablePassthrough
# Using RunnablePassthrough.assign to preserve data
enhanced_chain = RunnablePassthrough.assign(
    story=story_chain  
).assign(
    analysis=analysis_chain 
)
result = enhanced_chain.invoke({"topic": "a rainy day"})
print(result.keys())

dict_keys(['topic', 'story', 'analysis'])


In [21]:
result

{'topic': 'a rainy day',
 'story': 'The sky was a bruised plum, heavy and low, when Elara woke. A fat raindrop, daring and bold, splattered against her windowpane, followed by a chorus of its brethren. It was a day for staying inside, a day for the cozy embrace of blankets and the murmur of the rain orchestra outside.\n\nElara, a self-proclaimed sun worshipper, usually dreaded days like this. But today, a strange contentment settled over her. She stretched, listening to the rhythmic drumming on the roof, and felt a sense of permission to simply *be*.\n\nDownstairs, the aroma of cinnamon and damp earth filled the air. Her grandmother, Nana Rose, was already in the kitchen, humming a wordless tune as she stirred a pot on the stove.\n\n"Morning, sleepyhead," Nana Rose said, her voice warm and crinkled like old parchment. "Rainy day stew, your favorite."\n\nElara smiled. Nana Rose\'s rainy day stew was legendary. A hearty concoction of lentils, vegetables, and secret spices that always see

In [22]:
from operator import itemgetter
# Alternative approach using dictionary construction
manual_chain = (
RunnablePassthrough() | # Pass through input
{
"story": story_chain, # Add story result
"topic": itemgetter("topic") # Preserve original topic
} |
RunnablePassthrough().assign( # Add analysis based on story
analysis=analysis_chain
)
)
result = manual_chain.invoke({"topic": "a rainy day"})
print(result.keys()) # Output: dict_keys(['story', 'topic', 'analysis'])

dict_keys(['story', 'topic', 'analysis'])


In [23]:
result

{'story': 'The rain started as a whisper, a delicate drumming on the windowpane that woke Elara before her alarm. She burrowed deeper under the quilt, a worn patchwork of blues and greens, hoping the sound would fade. But the whisper quickly escalated into a steady, insistent downpour.\n\nElara groaned and peeled back the covers. The room was painted in shades of grey, the light filtered and muted by the thick curtain of rain outside. It was the kind of rain that promised to last all day, the kind that seeped into your bones and made you crave hot tea and a good book.\n\nShe shuffled downstairs, the old wooden floorboards creaking a mournful melody. The aroma of freshly brewed coffee already filled the air, a comforting beacon in the gloomy morning. Her grandmother, Nana Rose, was humming softly as she poured a steaming mug.\n\n“Morning, sleepyhead,” Nana Rose said, her eyes crinkling at the corners. “Looks like a perfect day for staying inside.”\n\nElara nodded, gratefully accepting t